In [1]:
import pandas as pd
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# sys.argv[1] = 'ratings.jsonl'
# sys.argv[2] = 'content.jsonl'
# sys.argv[3] = 'targets.csv'

Ratings = pd.read_json('ratings.jsonl', lines=True)
Content = pd.read_json('content.jsonl', lines=True)
Target = pd.read_csv('targets.csv')

submission = pd.DataFrame(columns=['UserId','ItemId'])


In [3]:
corpus = Content['Director'] + ' ' + Content['Genre'] + ' ' + Content['Year'] + ' ' + Content['Rated']


In [4]:
UsersRating = Ratings.copy()
UsersRating = UsersRating[UsersRating['UserId'].isin(Target['UserId'])]
UsersRating = UsersRating.reset_index(drop=True)

Users = Target['UserId'].copy()
Users = Users.drop_duplicates()
Users = Users.reset_index(drop=True)

Items = Target["ItemId"].copy()
Items = Items.drop_duplicates()
Items = Items.reset_index(drop=True)

items = pd.Series(Items.index,index=Items)
users = pd.Series(Users.index,index=Users)

In [5]:
len(Users)

6162

In [6]:
tf= TfidfVectorizer(stop_words='english')
matrix = tf.fit_transform(corpus)

In [7]:
cosine_sim = cosine_similarity(matrix,matrix)

In [8]:
a = Ratings[Ratings['Rating'] == Ratings['Rating'].max()]

In [71]:
result = pd.DataFrame(columns=['UserId','ItemId'])
count = 0
for U in Users:
    sims = []
    
    df = Ratings[Ratings['UserId'] == U]
    df = df[df['Rating'] == df['Rating'].max()]
    
    item = df.iloc[0]['ItemId']
    df2 = Target[Target['UserId'] == U]
    
    for idx,row in df2.iterrows():
        sims.append(cosine_sim[items[row['ItemId']]][items[item]])
    
    sims = sorted(enumerate(sims),key=lambda item: -item[1])
    idx = []
    for i in range(100):
        idx.append(sims[i][0])    
    
    new = df2.iloc[idx]
    result = result.append(new)
    count+=1
    print(count)
    
    
result.to_csv("submission.csv", index=False)

        UserId      ItemId
56  0006246bee  94aa5242b4
63  0006246bee  b273133e05
80  0006246bee  d5a93fa9c5
81  0006246bee  d9ed13c2f3
10  0006246bee  16ec9e966c
..         ...         ...
83  0006246bee  db2ecebeba
84  0006246bee  dcba99a1a6
86  0006246bee  dec4ce1bd7
91  0006246bee  e9c1be744c
97  0006246bee  f0871f1b79

[100 rows x 2 columns]
1
         UserId      ItemId
56   0006246bee  94aa5242b4
63   0006246bee  b273133e05
80   0006246bee  d5a93fa9c5
81   0006246bee  d9ed13c2f3
10   0006246bee  16ec9e966c
..          ...         ...
186  001fca6b30  d1cafe5463
189  001fca6b30  df0a89ee5d
196  001fca6b30  ee48cfc11e
197  001fca6b30  f3360c27f7
198  001fca6b30  f5f994eca0

[200 rows x 2 columns]
2
         UserId      ItemId
56   0006246bee  94aa5242b4
63   0006246bee  b273133e05
80   0006246bee  d5a93fa9c5
81   0006246bee  d9ed13c2f3
10   0006246bee  16ec9e966c
..          ...         ...
278  0038c2d256  c556fb254a
281  0038c2d256  c8c2df15aa
286  0038c2d256  d9d4de6c31
291  003

KeyboardInterrupt: 

In [ ]:
def get_rec():
    
    sim_scores = list(enumerate(cosine_sim[movie_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)


0.7257595942179731

In [ ]:
submission.to_csv('submission.csv',index=False)